In [10]:
import numpy as np
import pandas as pd
import os
import concurrent.futures
from random import random

#Pass path and tic of normalized data
#Splits data into testing and training dataframes
def splitCSVData(path,tic):
    #if path exists
    if os.path.exists( path + tic + r'.csv'):
        #read into pandas dataframe
        datav = pd.read_csv(path + tic + r'.csv', index_col=False)
        x= len(datav)
        rows=[r for r in range(0,x)]
        #grab the day to day change
        cols=["date", "dayToDay"] 
        dataframe=pd.DataFrame(data=datav, index=rows, columns=cols)
        #create frames for training and testing
        dataframeForTraining=pd.DataFrame(columns=cols)
        dataframeForTesting=pd.DataFrame(columns=cols)
        for k in range(0,x):
            val= random()
            row=dataframe.iloc[[k]]
            #send some of the data to testing
            if(val<.3):
                dataframeForTesting=dataframeForTesting.append(row,ignore_index=True)
            #send most to training
            else:
                dataframeForTraining=dataframeForTraining.append(row,ignore_index=True)
        return [dataframeForTesting, dataframeForTraining]
                
#Pass path and tic to normalized data
#exports outoput of splitCSVData to separate csvs
def exportToCSVTestingAndTraining(path,tic):
    #if the path exists
    if os.path.exists(path + tic + r'.csv'):
        #see if we already have the file
        if os.path.exists(r'testing/' + tic + r'.csv'):
            print("already chillin")
        #if we do not
        else:
            #split the data
            dataSplit=splitCSVData(path,tic)
            #export to respective folders
            testing=dataSplit[0].tail(3500)
            training=dataSplit[1].tail(3500)
            testing.to_csv(r'testing/' + tic + r'.csv',  index = False)
            training.to_csv(r'training/' + tic + r'.csv', index = False) 




In [4]:
import numpy as np
import pandas as pd
import os
import concurrent.futures

#get the percentage change of value relative to base
def percentChange(base, value):
    if(base!=0):
        diff=value-base
        div=diff/base
        div+=1
        return div
    else:
        raise ZeroDivisionError
def transform(vals):
    v=[[]]
    alpha=len(vals[0])-1
    while(alpha>0):
        v.append([])
        alpha-=1
    for k in range(0,len(vals[0])):
        for j in range(0,len(vals)):
            v[k].append(vals[j][k])
    return v
#normalizes all the data via percentage change with 
#a base as the 52 day average
def normalize(path, tic):
    if os.path.exists( path + tic + r'.csv'):
        print('start: ' + tic)
        data = pd.read_csv(path + tic + r'.csv')
        low= [0]*len(data['low'])
        high= [0]*len(data['low'])
        average= [0]*len(data['low'])
        volume= [0]*len(data['low'])
        close= [0]*len(data['low'])
        openn= [0]*len(data['low'])
        rangee= [0]*len(data['low'])
        twelveDay= [0]*len(data['low'])
        twentySixDay= [0]*len(data['low'])
        volumeEMA= [0]*len(data['low'])
        singleDay= [0]*len(data['low'])
        dayToDay= [0]*len(data['low'])
        fiftyTwoDayHigh= [0]*len(data['low'])
        fiftyTwoWeekHigh= [0]*len(data['low'])
        fiftyTwoDayLow= [0]*len(data['low'])
        fiftyTwoWeekLow= [0]*len(data['low'])
        fiftyTwoWeekAverage= [0]*len(data['low'])
        fiftyTwoDayStandDev= [0]*len(data['low'])
        fiftyTwoWeekStandDev= [0]*len(data['low'])
        for i in range(0,len(data['high'])):
            base=data['52 day average'][i]
            baseVol= data['52 week volume average'][i]
            if(baseVol==0):
                baseVol=1
            low[i]=percentChange(base, data['low'][i])
            high[i]=percentChange(base, data['high'][i])
            average[i]=percentChange(base, data['average'][i])
            close[i]=percentChange(base, data['close'][i])
            openn[i]=percentChange(base, data['open'][i])
            rangee[i]=percentChange(base, data['range'][i])
            twelveDay[i]=percentChange(base, data['12 day ema'][i])
            twentySixDay[i]=percentChange(base, data['26 day ema'][i])
            volume[i]=percentChange(baseVol,data['volume'][i])
            volumeEMA[i]=percentChange(baseVol,data['volume ema'][i])
            singleDay[i]=percentChange(base, data['single_day_change'][i])
            dayToDay[i]=percentChange(base, data['day_to_day_change'][i])
            fiftyTwoDayHigh[i]=percentChange(base, data['52 day high'][i])
            fiftyTwoWeekHigh[i]=percentChange(base, data['52 week high'][i])
            fiftyTwoDayLow[i]=percentChange(base, data['52 day low'][i])
            fiftyTwoWeekLow[i]=percentChange(base, data['52 week low'][i])
            fiftyTwoWeekAverage[i]=percentChange(base, data['52 week average'][i])
            fiftyTwoDayStandDev[i]=percentChange(base, data['52 day standard deviation'][i])
            fiftyTwoWeekStandDev[i]=percentChange(base, data['52 week standard deviation'][i])
    else:
        return -1
    listOfDates= data['date']
    listOfVols= data['52 week volume average']
    listOfAverages= data['52 day average']
    vals= [low, high, average, volume, close, openn, rangee, twelveDay, twentySixDay,volumeEMA, singleDay, dayToDay,fiftyTwoDayHigh,fiftyTwoWeekHigh,fiftyTwoDayLow,fiftyTwoWeekLow,fiftyTwoWeekAverage,fiftyTwoDayStandDev,fiftyTwoWeekStandDev]  
    v=transform(vals)
    rows=listOfDates
    cols=["low", "high", "average", "volume", "close", "open", "range", "twelveDay", "twentySixDay","volumeEMA", "singleDay", "dayToDay","fiftyTwoDayHigh","fiftyTwoWeekHigh","fiftyTwoDayLow","fiftyTwoWeekLow","fiftyTwoWeekAverage","fiftyTwoDayStandDev","fiftyTwoWeekStandDev"] 
    dataframe=pd.DataFrame(data=v, index=rows, columns=cols)
    return dataframe


getPath = 'C:/Users/csstudent/stock-project-master/historical_stock_data/historical_stock_data/'
savePath = 'C:/Users/csstudent/stock-project-master/stock-project-master/data/normalized_data/'

def update_stock(tic):
    if(os.path.exists(savePath+tic+r'.csv')):
        print("check")
    else:
        data = normalize(getPath, tic)
        if not isinstance(data, int):
            data.to_csv(savePath + tic + r'.csv', index = True)
            print('end: ' + tic)
        return(1)


In [5]:
#for i in range(0,len(tickers)):
    #update_stock(tickers[i])

In [8]:
#all the stock tickers
tickers = pd.read_csv('stock_names.csv')['Ticker'] #gets stock Tickers 
tickers

0          A
1        AAC
2       AACG
3        AAL
4       AAME
        ... 
5116    ZUMZ
5117     ZUO
5118     ZVO
5119    ZYNE
5120    ZYXI
Name: Ticker, Length: 5121, dtype: object

In [11]:
#for every tic, export the training and testing data
#this is my specific path, have to change for yours
path='C:/Users/csstudent/stock-project-master/stock-project-master/data/normalized_data/'
for tic in range(0,len(tickers)):
    exportToCSVTestingAndTraining(path,tickers[tic])

already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
ABEO
already chillin
already chillin
already chillin
already chillin
ABR-A
ABR-C
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
ACER
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
ACST
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chillin
already chill

BSCE
BSCK
BSCL
BSCM
BSCN
BSCO
BSCP
BSCQ
BSCR
BSCS
BSCT
BSDE
BSE
BSET
BSGM
BSJK
BSJL
BSJM
BSJN
BSJO
BSJP
BSJQ
BSJR
BSL
BSML
BSMM
BSMN
BSMO
BSMP
BSMQ
BSMR
BSMS
BSMT
BSMX
BSQR
BSRR
BSTC
BSTZ
BSVN
BTA
BTAI
BTEC
BTI
BTT
BTZ
BUG
BUI
BURG
BUSE
BV
BVSN
BVXV
BVXVW
BW
BWAY
BWB
BWEN
BWFG
BWG
BWMC
BWMCU
BWMCW
BX
BXG
BXMX
BXP-B
BXRX
BY
BYFC
BYM
BYND
BYSI
BZM
BZUN
C
C-K
C-S
CAAS
CABA
CABO
CAC
CACC
CACG
CADE
CAF
CAH
CAI-A
CAJ
CAKE
CALA
CALM
CALX
CAMP
CAMT
CAN
CAPL
CAPR
CAR
CARA
CARE
CARG
CARO
CARV
CARZ
CASA
CASH
CASI
CASS
CASY
CAT
CATB
CATC
CATH
CATM
CATS
CATY
CB
CBAN
CBAT
CBAY
CBB-B
CBFV
CBH
CBIO
CBL-D
CBLI
CBM
CBMB
CBMG
CBNK
CBPO
CBRE
CBRL
CBS.A
CBSH
CBSHP
CBTX
CBU
CC
CCAP
CCB
CCBG
CCCL
CCD
CCEP
CCH.U
CCI
CCJ
CCL
CCLP
CCMP
CCNE
CCO
CCOI
CCRC
CCRN
CCS
CCX
CCX.W
CCXI
CDAY
CDC
CDEV
CDK
CDL
CDLX
CDMO
CDMOP
CDNA
CDNS
CDR
CDR-C
CDTX
CDW
CDXC
CDXS
CDZI
CEA
CECE
CEIX
CELC
CELH
CELP
CEMI
CEN
CENT
CENTA
CENX
CEPU
CEQP
CERC
CERN
CERS
CETV
CETX
CETXP
CETXW
CEVA
CEY
CEZ
CFA
CFB
CFBI
CFBK
CFFA
CFFAU
CFFAW
CFFI
C

IMKTA
IMMP
IMMR
IMMU
IMOS
IMRN
IMRNW
IMTE
IMUX
IMV
IMVT
IMVTU
IMVTW
IMXI
INAP
INBK
INBKL
INBKZ
INCY
INDB
INDY
INF
INFI
INFN
INFR
INFY
INGN
INGR
INMB
INMD
INN-D
INNT
INO
INOD
INOV
INPX
INS-A
INSE
INSG
INSM
INSP
INSU
INSUU
INSUW
INSW
INTC
INTG
INTL
INTU
INVA
INVE
INVH
INWK
IO
IONS
IOSP
IOTS
IOVA
IPAR
IPG
IPGP
IPHA
IPHS
IPI
IPKW
IPLDP
IPO.W
IPWR
IQ
IQI
IR
IRBT
IRCP
IRDM
IRET
IRIX
IRM
IRMD
IROQ
IRS
IRTC
IRWD
ISBC
ISD
ISDS
ISDX
ISEE
ISEM
ISHG
ISIG
ISNS
ISRG
ISSC
ISTB
ISTR
IT
ITCI
ITGR
ITI
ITIC
ITMR
ITRI
ITRM
ITRN
ITUB
IUS
IUSB
IUSG
IUSS
IUSV
IVAC
IVC
IVR
IVR-B
IVZ
IXUS
IZEA
JACK
JAG
JAGX
JAKK
JAN
JASN
JAZZ
JBGS
JBHT
JBL
JBLU
JBR
JBSS
JCA-B
JCE
JCO
JCOM
JCS
JCTCF
JD
JDD
JE-A
JEF
JEMD
JFIN
JFK
JFKKR
JFKKU
JFKKW
JFR
JFU
JG
JHAA
JHD
JHI
JHX
JILL
JJSF
JKHY
JKI
JLL
JMEI
JMIA
JMM
JMPB
JMPE
JMPNL
JMPNZ
JMU
JNCE
JNJ
JOBS
JOE
JOUT
JP
JPI
JPM-A
JPM-D
JPM-G
JPS
JQC
JRJC
JRO
JRSH
JRVR
JSD
JSM
JSMD
JSML
JTA
JVA
JW.A
JWN
JYNT
K
KALA
KALU
KALV
KAMN
KB
KBAL
KBLM
KBLMR
KBLMU
KBLMW
KBR
KBSF
KBWB
KBWD
KBWP
KBW

SCP.W
SCPH
SCPL
SCS
SCSC
SCVL
SCWX
SCX
SCYX
SCZ
SDC
SDG
SDR
SDT
SDVY
SEAC
SEAS
SECO
SEDG
SEED
SEEL
SEIC
SELB
SELF
SEM
SENEA
SENEB
SERV
SES
SESN
SF-A
SFB
SFBC
SFBS
SFET
SFIX
SFL
SFM
SFNC
SFST
SG
SGA
SGBX
SGC
SGEN
SGH
SGLB
SGLBW
SGMA
SGMO
SGMS
SGOC
SGRP
SGRY
SGU
SHBI
SHEN
SHG
SHIP
SHIPW
SHIPZ
SHL.U
SHLL
SHLO
SHO
SHO-F
SHOO
SHSP
SHV
SHW
SHY
SIBN
SIC
SIEB
SIEN
SIFY
SIG
SIGA
SIGI
SILC
SILK
SIMO
SINA
SINO
SINT
SIRI
SIT-J
SITC
SITM
SITO
SIVB
SIVBP
SIX
SJIJ
SJM
SJT
SKM
SKOR
SKX
SKYS
SKYW
SKYY
SLAB
SLB
SLCT
SLDB
SLF
SLG-I
SLGG
SLGL
SLGN
SLIM
SLM
SLMBP
SLNO
SLP
SLQD
SLRC
SLRX
SLS
SLVO
SMAR
SMBC
SMBK
SMCI
SMCP
SMED
SMG
SMH
SMIT
SMLP
SMMC
SMMCU
SMMCW
SMMF
SMMT
SMP
SMPL
SMRT
SMSI
SMTC
SMTX
SNA
SNBR
SNCA
SNCR
SND
SNDE
SNDL
SNDR
SNDX
SNES
SNFCA
SNGX
SNGXW
SNLN
SNN
SNOA
SNOAW
SNPS
SNR
SNSR
SNSS
SNV-D
SNX
SNY
SOCL
SOGO
SOHO
SOHOB
SOHON
SOHOO
SOHU
SOJA
SOJC
SOLN
SOLO
SOLOW
SOLY
SONA
SONG
SONGW
SONM
SONO
SOR
SORL
SOXX
SP
SPA.W
SPAR
SPB
SPCB
SPE-B
SPEX
SPFI
SPG-J
SPH
SPHS
SPI
SPKE
SPKEP
SPLK
SPLP
SPNE
SPN